In [1]:
import pandas as pd
from IPython.display import Markdown
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_selection import VarianceThreshold
import re

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk import tokenize

In [3]:
nltk.download('rslp')

[nltk_data] Downloading package rslp to /home/cristina/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [4]:
token_space = tokenize.WhitespaceTokenizer()

In [5]:
df = (
    pd
    .read_csv('../data/external/data.csv')
)
df = df.assign(
    site = df.site.str.split("/").apply(lambda x: f"{x[0]}//{x[2]}")
)
display(Markdown("### Conjunto de dados"))
display(df.head())

df_dict = pd.read_csv('../data/external/dicionario.csv')
display(Markdown("### Dicionário de dados"))
display(df_dict.head())

### Conjunto de dados

,text,datetime,category,label,site
0,Kátia Abreu diz que vai colocar sua expulsão e...,2017-11-30,politica,fake,https://ceticismopolitico.com
1,"Dr. Ray peita Bolsonaro, chama-o de conservad...",2017-11-24,politica,fake,https://ceticismopolitico.com
2,Reinaldo Azevedo desmascarado pela Polícia Fed...,2017-05-23,politica,fake,https://afolhabrasil.com.br
3,Relatório assustador do BNDES mostra dinheiro ...,24/07/2017,politica,fake,https://www.diariodobrasil.org
4,"Radialista americano fala sobre o PT: ""Eles ve...",25/07/2017,politica,fake,https://www.diariodobrasil.org


### Dicionário de dados

,variavel,significado,tipo,valores
0,text,"Coluna contendo o texto das matérias, sendo ca...",text,string
1,datetime,Coluna contendo a data de publicação das matérias,time,datetime
2,category,Coluna contetndo a categoria na qual cada maté...,nominal,"['politica', 'tv_celebridades', 'sociedade_cot..."
3,label,Coluna contendo os rótulos se a notícia é fals...,nominal,"['fake', 'true']"
4,site,Coluna contendo o link do site em que a notíci...,nominal,string


In [6]:
# cria dados sem pontuação, normalizado (ideal)
df_clean = df.drop(columns = ["datetime"])
df_clean["text"]=df_clean["text"].apply(lambda x : x.lower())
df_clean["text"]=df_clean["text"].apply(lambda x : re.sub('[^\w\s]', '', x))
display(df_clean)

,text,category,label,site
0,kátia abreu diz que vai colocar sua expulsão e...,politica,fake,https://ceticismopolitico.com
1,dr ray peita bolsonaro chamao de conservador f...,politica,fake,https://ceticismopolitico.com
2,reinaldo azevedo desmascarado pela polícia fed...,politica,fake,https://afolhabrasil.com.br
3,relatório assustador do bndes mostra dinheiro ...,politica,fake,https://www.diariodobrasil.org
4,radialista americano fala sobre o pt eles vend...,politica,fake,https://www.diariodobrasil.org
...,...,...,...,...
7195,para jornal britânico ação contra lula na lava...,politica,true,http://politica.estadao.com.br
7196,temer diz que acionou pf e cade para investiga...,politica,true,https://g1.globo.com
7197,os obstáculos políticos de temer em 2017 espec...,politica,true,https://g1.globo.com
7198,sextafeira 15 de setembro de 2017 boa noite aq...,sociedade_cotidiano,true,https://g1.globo.com


In [7]:
df_clean_no_stopWords = df_clean.copy()
df_clean_with_stopWords = df_clean.copy()

### DF sem stopwords

In [8]:
palavras_irrelevantes = set(nltk.corpus.stopwords.words("portuguese"))

In [9]:
df_clean_no_stopWords.head(2)

,text,category,label,site
0,kátia abreu diz que vai colocar sua expulsão e...,politica,fake,https://ceticismopolitico.com
1,dr ray peita bolsonaro chamao de conservador f...,politica,fake,https://ceticismopolitico.com


In [10]:
frase_processada = list()
for text in df_clean_no_stopWords["text"]:
    #para cada frase acessada vamos tokeniza-la e verificar se cada token 
    #é pertecentes a nossa lista de stopwords
    nova_frase = list()
    palavras_texto = token_space.tokenize(text)
    for palavra in palavras_texto:
        #para cada palavra vamos verificar se a mesma esta na nossa lista de 
        #palavras_irrelevantes
        if palavra not in palavras_irrelevantes:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))

df_clean_no_stopWords["text"]=frase_processada


In [11]:
display(df_clean_no_stopWords.head(2))

,text,category,label,site
0,kátia abreu diz vai colocar expulsão moldura r...,politica,fake,https://ceticismopolitico.com
1,dr ray peita bolsonaro chamao conservador fake...,politica,fake,https://ceticismopolitico.com


#### Stemmatização

In [12]:
df_no_stopWords_stemma = df_clean_no_stopWords.copy()

In [13]:
stemmer = nltk.stem.RSLPStemmer()

In [14]:
frase_processada = list()
for text in df_no_stopWords_stemma["text"]:
    #para cada frase acessada vamos tokeniza-la e verificar se cada token 
    #é pertecentes a nossa lista de stop
    #words
    nova_frase = list()
    palavras_texto = token_space.tokenize(text)
    for palavra in palavras_texto:
        nova_frase.append(stemmer.stem(palavra)) #stemmatização de cada palavra da sentença
    frase_processada.append(' '.join(nova_frase))

df_no_stopWords_stemma["text"]=frase_processada

In [15]:
df_no_stopWords_stemma

,text,category,label,site
0,kát abr diz vai coloc expuls mold reclam sen k...,politica,fake,https://ceticismopolitico.com
1,dr ray peit bolsonar chama conserv fak entrev ...,politica,fake,https://ceticismopolitico.com
2,reinald azeved desmascar políci feder ferrenh ...,politica,fake,https://afolhabrasil.com.br
3,relatóri assust bnd mostr dinh públic brasil j...,politica,fake,https://www.diariodobrasil.org
4,radial americ fal sobr pt vend ilusã pov brasi...,politica,fake,https://www.diariodobrasil.org
...,...,...,...,...
7195,jorn britân açã contr lul lav jat julg sécul a...,politica,true,http://politica.estadao.com.br
7196,tem diz acion pf cad investig aument preç gaso...,politica,true,https://g1.globo.com
7197,obstácul polít tem 2017 especi ouv g1 aval alé...,politica,true,https://g1.globo.com
7198,sextaf 15 setembr 2017 boa noit aqu princip no...,sociedade_cotidiano,true,https://g1.globo.com


### Lemmatização

In [16]:
import spacy
nlp = spacy.load('pt_core_news_sm')

In [19]:
df_no_stopWords_lemma = df_clean_no_stopWords.copy()

In [20]:
frase_processada = list()
for text in df_no_stopWords_lemma["text"]:
    nova_frase = list()
    doc= nlp(text)
    for palavra in doc:
        
        nova_frase.append(palavra.lemma_) #lemmatização de cada palavra da sentença
    frase_processada.append(' '.join(nova_frase))

df_no_stopWords_lemma["text"]=frase_processada

In [21]:
df_no_stopWords_lemma.text[1]

'dr Ray peita bolsonaro chamao conservador fake entrevista danilo gentili dividir direita site vir avisar jair bolsonaro dever abandonar pauta estatisto vez fazer discurso convincente bom parte liberal conservador brasil querer ver livre amarra estatal tudo bem pesquiso ainda dizer grande parte povo contra privatização índice pouco 50 povo fácil reverter ademal bolsonaro dever falar direitisto vez focar tanto petista arrependir recentemente dizer pensar 200 vez antes privatizar petrobrás cair mão chinês algo tipo dever ter dizer garanto privatização petrobrás Garanto chinês ir comprála deixar brecha jeito falar parecer suposto medo venda chinês pretexto evitar privatização lá direita ir ter adotar alternativa foquar estado reduzir diminuição imposto venda estatal além João amoedo dr rey fazer vicejar tipo discurso ainda candidaturar fase inicial complicado bolsonaro apareçar pessoa direita propor visão economicamente direitista economia enfim ver 3240 dr rey espinafrar bolsonaro dar br

In [22]:
df_no_stopWords_stemma.text[1]

'dr ray peit bolsonar chama conserv fak entrev danil gentil divid direit sit vem avis jair bolsonar dev abandon paut estat vez faz discurs convinc boa part liberal conserv brasil quer ver livr amarr estatal tud bem pesquis aind diz mai part pov contr priva índic pouc 50 pov fácil revert adem bolsonar dev fal direit vez foc tant petist arrepend recent diss pens 200 vez ant priva petrobr cai mão chines alg tip dev ter dit garant priva petrobr garant chines irã comprál deix brech jeit fal parec supost med vend chines pretext evit priva lá direit vai ter adot altern foqu est reduz diminu impost vend estatal além joã amoed dr rey faz vicej tip discurs aind candidat fas inic complic bolsonar apareç pesso direit prop vis economic direit econom enfim vej 3240 dr rey espinafr bolsonar dá brech pod reclam outr aproveit'

### DF sem stopwords

stemma

In [23]:
df_clean_with_stopWords_stemma = df_clean_with_stopWords.copy()

In [24]:
frase_processada = list()
for text in df_clean_with_stopWords_stemma["text"]:
    #para cada frase acessada vamos tokeniza-la e verificar se cada token 
    #é pertecentes a nossa lista de stop
    #words
    nova_frase = list()
    palavras_texto = token_space.tokenize(text)
    for palavra in palavras_texto:
        nova_frase.append(stemmer.stem(palavra)) #stemmatização de cada palavra da sentença
    frase_processada.append(' '.join(nova_frase))

df_clean_with_stopWords_stemma["text"]=frase_processada

In [25]:
df_clean_with_stopWords_stemma

,text,category,label,site
0,kát abr diz que vai coloc sua expuls em uma mo...,politica,fake,https://ceticismopolitico.com
1,dr ray peit bolsonar chama de conserv fak em e...,politica,fake,https://ceticismopolitico.com
2,reinald azeved desmascar pel políci feder o ma...,politica,fake,https://afolhabrasil.com.br
3,relatóri assust do bnd mostr dinh públic do br...,politica,fake,https://www.diariodobrasil.org
4,radial americ fal sobr o pt ele vend ilusã ao ...,politica,fake,https://www.diariodobrasil.org
...,...,...,...,...
7195,par jorn britân açã contr lul na lav jat ser o...,politica,true,http://politica.estadao.com.br
7196,tem diz que acion pf e cad par investig aument...,politica,true,https://g1.globo.com
7197,os obstácul polít de tem em 2017 especi ouv pe...,politica,true,https://g1.globo.com
7198,sextaf 15 de setembr de 2017 boa noit aqu est ...,sociedade_cotidiano,true,https://g1.globo.com


Lemma

In [26]:
df_clean_with_stopWords_lemma = df_clean_with_stopWords.copy()

In [27]:
frase_processada = list()
for text in df_clean_with_stopWords_lemma["text"]:
    nova_frase = list()
    doc= nlp(text)
    for palavra in doc:
        
        nova_frase.append(palavra.lemma_) #lemmatização de cada palavra da sentença
    frase_processada.append(' '.join(nova_frase))

df_clean_with_stopWords_lemma["text"]=frase_processada

In [28]:
df_clean_with_stopWords_lemma

,text,category,label,site
0,kátia abrer dizer que ir colocar seu expulsão ...,politica,fake,https://ceticismopolitico.com
1,dr Ray peita bolsonaro chamao de conservador f...,politica,fake,https://ceticismopolitico.com
2,reinaldo azevedo desmascarar por o polícia fed...,politica,fake,https://afolhabrasil.com.br
3,relatório assustador de o bnde mostrar dinheir...,politica,fake,https://www.diariodobrasil.org
4,radialista americano falar sobre o pt eles ven...,politica,fake,https://www.diariodobrasil.org
...,...,...,...,...
7195,para jornal britânico ação contra Lula em o la...,politica,true,http://politica.estadao.com.br
7196,temer dizer que acionar pf e cade para investi...,politica,true,https://g1.globo.com
7197,o obstáculo político de temer em 2017 especial...,politica,true,https://g1.globo.com
7198,sextafeira 15 de setembro de 2017 bom noite aq...,sociedade_cotidiano,true,https://g1.globo.com


In [29]:
path = "../data/processed/"

In [30]:
df_no_stopWords_lemma.to_csv(path+"no_stopWords_lemma.csv",index=False)
df_no_stopWords_stemma.to_csv(path+"no_stopWords_stemma.csv",index=False)

In [35]:
df_clean_with_stopWords_lemma.to_csv(path+"with_stopWords_lemma.csv",index=False)
df_clean_with_stopWords_stemma.to_csv(path+"with_stopWords_stemma.csv",index=False)